## Exercise 04. A/B-testing ##

## Imports ##

In [6]:
import pandas as pd
import sqlite3

## Task ##
## Test and control results ##
1. Создайте подключение к базе данных с использованием библиотеки `sqlite3`.
2. Используя только один запрос для каждой группы, создайте два датафрейма: `test_results` и `control_results`, с колонками "time" и "avg_diff", и по две строки.
   - "time" должны содержать значения "after" и "before".
   - "avg_diff" должны содержать среднюю дельту для всех пользователей за периоды до и после их первого посещения страницы.
   - При этом учитывать только пользователей, у которых есть наблюдения и до, и после этого события.
3. Не учитывайте лабораторную работу `project1`.
4. Закройте соединение.

In [7]:
conn = sqlite3.connect('../data/checking-logs.sqlite')

In [8]:
query = '''
WITH users_before AS (
    SELECT t.uid,
           AVG((t.deadlines - strftime('%s', t.first_commit_ts)) / 3600.0) AS avg_delta_hours
    FROM (
        SELECT uid, deadlines, first_commit_ts, first_view_ts, labname
        FROM test
        JOIN deadlines ON test.labname = deadlines.labs
        WHERE labname != 'project1'
          AND first_commit_ts < first_view_ts
    ) t
    GROUP BY t.uid
), users_after AS (
    SELECT t.uid,
           AVG((t.deadlines - strftime('%s', t.first_commit_ts)) / 3600.0) AS avg_delta_hours
    FROM (
        SELECT uid, deadlines, first_commit_ts, first_view_ts, labname
        FROM test
        JOIN deadlines ON test.labname = deadlines.labs
        WHERE labname != 'project1'
          AND first_commit_ts >= first_view_ts
    ) t
    GROUP BY t.uid
), common_users AS (
    SELECT uid FROM users_before
    INTERSECT
    SELECT uid FROM users_after
)

SELECT 'before' AS time, AVG(avg_delta_hours) AS avg_diff
FROM users_before 
WHERE uid IN (SELECT uid FROM common_users)

UNION ALL

SELECT 'after' AS time, AVG(avg_delta_hours) AS avg_diff
FROM users_after
WHERE uid IN (SELECT uid FROM common_users)
'''

test_results = pd.read_sql(query, conn)
print(test_results)

     time    avg_diff
0  before   66.679583
1   after  100.178181


In [9]:
query = '''
WITH users_before AS (
    SELECT t.uid,
           AVG((t.deadlines - strftime('%s', t.first_commit_ts)) / 3600.0) AS avg_delta_hours
    FROM (
        SELECT uid, deadlines, first_commit_ts, first_view_ts, labname
        FROM control
        JOIN deadlines ON control.labname = deadlines.labs
        WHERE labname != 'project1'
          AND first_commit_ts < first_view_ts
    ) t
    GROUP BY t.uid
), users_after AS (
    SELECT t.uid,
           AVG((t.deadlines - strftime('%s', t.first_commit_ts)) / 3600.0) AS avg_delta_hours
    FROM (
        SELECT uid, deadlines, first_commit_ts, first_view_ts, labname
        FROM control
        JOIN deadlines ON control.labname = deadlines.labs
        WHERE labname != 'project1'
          AND first_commit_ts >= first_view_ts
    ) t
    GROUP BY t.uid
), common_users AS (
    SELECT uid FROM users_before
    INTERSECT
    SELECT uid FROM users_after
)

SELECT 'before' AS time, AVG(avg_delta_hours) AS avg_diff
FROM users_before 
WHERE uid IN (SELECT uid FROM common_users)

UNION ALL

SELECT 'after' AS time, AVG(avg_delta_hours) AS avg_diff
FROM users_after
WHERE uid IN (SELECT uid FROM common_users)
'''

control_results = pd.read_sql(query, conn)
print(control_results)

     time   avg_diff
0  before  98.467852
1   after  99.803549


In [10]:
conn.close()

## Conclusion ##
5. Вопрос: оказалась ли гипотеза верной и влияет ли просмотр страницы на поведение студентов?

## Ответ: влияет ##
Но только отрицательно